In [5]:
import httplib2
import oauth2
import urllib3
import types
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gender_detector import GenderDetector 
import psycopg2, psycopg2.extras
from causalinference import CausalModel
from causalinference.utils import random_data
import httplib
import base64
import json # For Microsoft Face API
import urllib as urllib # For Microsoft Face API
import time 
import csv
import datetime 


class UpworkDataFormatter:
    
    def __init__(self):
        # Settings
        self.present_date = "12/2017" # This is the month in which the data was collected
        self.all_data_file_name = './csv_files/debug_2017_12_12_upwork_analysis_unitedstates_allskills.csv' # Filename for all data
        self.face_data_file_name = './csv_files/2017_12_12_upwork_face_data_unitedstates_allskills.csv' # Filename for gender data
        self.data_log_file_name = './log_files/debug_log_upwork_data_analysis_2017_12_12_unitedstates_allskills.txt'
        self.face_log_file_name = "./log_files/log_upwork_face_data_2017_12_12_unitedstates_allskills.txt"
        self.github_photo_path = 'https://raw.githubusercontent.com/efoongch/upwork-pay-by-gender/master/resized_profile_photos/resized_images_unitedstates_allskills_2017_12_12/'
        self.face_api_key = "" # Will ask for key raw input later on 
        
        # Write a log
        self.log = open(self.data_log_file_name, 'a')
        self.photo_log = open(self.face_log_file_name, 'a')
        self.log.write("We have started analyzing data!" + "\n")
        self.log.flush()_log.write("We have started saving face data from photos!" + "\n")
        self.photo_log.flush()

        # Connect to the database 
        self.conn = psycopg2.connect("dbname=eureka01")
        self.cur = self.conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        psycopg2.extensions.register_adapter(dict, psycopg2.extras.Json)
        
        # Get detailed_info from workers in our database
        self.cur.execute("SELECT detailed_info FROM upwork_unitedstates_allskills_2017_12_12;")
        
        # Initialize arrays for Causal Analysis 
        self.user_count = 1
        self.bill_rate_array = []
        self.gender_array = []
        self.all_covariates_array = []
        self.country_id_list = []
        self.job_category_id_list = []
        self.education_id_list = []
        self.error_country_id = 0
        self.error_education_id = 0
        self.error_job_category_id = 0
        self.no_edu_history_count = 0
        self.schools_no_degree = []
        self.ambiguous_edu_count = 0
        self.list_of_ambiguous_degrees = []        
        self.other_info = []
        self.diploma_info = []
        self.ambiguous_info = []
        self.none_specified_info = []
        self.empty_info = []
        self.high_school_degrees = []
    
    def save_all_to_csv(self):me, 'w') as csvfile:
            fieldnames = ['user_count','worker_id',_name', 'bill_rate', 
                          'country', 'country_id', 'degree', 'education', 'education_id', 
                          'work_experience', 'job_category', 'job_category_id', 'city', 
                          'timezone', 'date_last_active', 'jobs_completed',
                         'profile_desc']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            for user in self.cur:
                try: 
                    user_count = self.user_count
                    worker_id = user[0]["ciphertext"]
                    first_name = user[0]["dev_first_name"].encode('utf-8').strip()
                    bill_rate = user[0]["dev_bill_rate"]
                    country = user[0]["dev_country"]
                    country_id = self.encode_category(country, self.country_id_list)
                    degree = self.show_degree(user)
                    education = self.calculate_education(user)
                    if (education == "No history"):
                        self.no_edu_history_count += 1
                    education_id = self.encode_category(education, self.education_id_list)
                    work_experience = self.calculate_work_experience(user)
                    job_category = self.identify_job_category(user)
                    job_category_id = self.encode_category(job_category, self.job_category_id_list)
                    city = user[0]['dev_city']
                    timezone = user[0]['dev_timezone']
                    date_last_active = user[0]['dev_last_activity']
                    jobs_completed = user[0]['dev_billed_assignments'] # Number of billed assignments 
                    profile_desc = user[0]['dev_blurb']
                    
                    writer.writerow({'user_count': self.user_count, 'worker_id': worker_id, 'first_name': first_name,
                                 'bill_rate': bill_rate, 'country': country, 'country_id': country_id, 'degree': degree, 'education': education, 
                                 'education_id': education_id, 'work_experience': work_experience, 'job_category': job_category, 'job_category_id': job_category_id, 
                                    'city': city, 'timezone': timezone, 'date_last_active': date_last_active, 
                                    'jobs_completed': jobs_completed, 
                                    'profile_desc': profile_desc})
                except:
                    print "Ran into some error at user {0}".format(self.user_count)
                    
                    writer.writerow({'user_count': self.user_count, 'worker_id': "error", 'first_name': "error",
                                 'bill_rate': "error", 'country': "error", 'country_id': self.error_country_id, 'degree': "error", 'education': "error", 
                                 'education_id': self.error_education_id, 'work_experience': "error", 'job_category': "error", 'job_category_id': self.error_job_category_id, 
                                    'city': "error", 'timezone': "error", 'date_last_active': "error", 
                                    'jobs_completed': "error", 
                                    'profile_desc': "error"})
                    
                print "Finished writing data for {0}".format(self.user_count)
                self.user_count += 1
                
        print "This is the total workers without education history: {0}".format(self.no_edu_history_count)
        
        print "These are all the schools with no degree specified: {0}".format(self.schools_no_degree)
        
        with open("schools_without_degree.csv", 'w') as csvfile:
            school_writer = csv.DictWriter(csvfile, fieldnames=['school'])
            school_writer.writeheader()
            
            for school in self.schools_no_degree:
                school_writer.writerow({'school': school.encode('utf-8')})
        
        # For debugging education
        file = open("ambiguous_education_2.txt", "w")
        for item in self.list_of_ambiguous_degrees:
            file.write(item.encode('utf-8') + '\n')
        file.close()
        
        print "Total ambiguous education people in dataset: {0}".format(self.ambiguous_edu_count)
        
        # Debugging education entries with "diploma" or "other"
        with open("other_diploma_debugging.csv", 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=['degree_category', 'institution', 'comment', 'degree'])
            writer.writeheader()

            for item in self.other_info:
                writer.writerow({'degree_category': 'Other', 'institution': item[0].encode('utf-8'), 
                                 'comment': item[1].encode('utf-8'), 
                                'degree': item[2].encode('utf-8')})
            
            for item in self.diploma_info:
                writer.writerow({'degree_category': 'Diploma', 'institution': item[0].encode('utf-8'), 
                                 'comment': item[1].encode('utf-8'), 
                                'degree': item[2].encode('utf-8')})
        
            for item in self.ambiguous_info:
                writer.writerow({'degree_category': 'Ambiguous', 'institution': item[0].encode('utf-8'), 
                                 'comment': item[1].encode('utf-8'), 
                                'degree': item[2].encode('utf-8')})
                
            for item in self.none_specified_info:
                writer.writerow({'degree_category': 'None', 'institution': item[0].encode('utf-8'), 
                                 'comment': item[1].encode('utf-8'), 
                                'degree': item[2].encode('utf-8')})
                
            for item in self.empty_info:
                writer.writerow({'degree_category': 'Empty', 'institution': item[0].encode('utf-8'), 
                                 'comment': item[1].encode('utf-8'), 
                                'degree': item[2].encode('utf-8')})
                
        # Debugging high school entries
        with open("high_school_debugging.csv", 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=['institution', 'comment', 'degree'])
            writer.writeheader()

            for item in self.high_school_degrees:
                writer.writerow({'institution': item[0].encode('utf-8'), 
                                 'comment': item[1].encode('utf-8'), 
                                'degree': item[2].encode('utf-8')})

    def check_state(city): # This code doesn't work; temporarily suspending this until better method found
        city_array = city.split()
        city_length = len(city_array)
        possible_matches = []

        with open('2017_Gaz_place_national.txt', 'r') as state_data:
            next(state_data)
            for line in state_data:
                line_array = line.split()
                city_data = line_array[3]

                # Look for exact match in the dataset 
                if re.search(city, line):
                    if (line_array[3 + city_length].islower() or line_array[3 + city_length] == "CDP"):
                        possible_matches.append(line_array)

        if len(possible_matches) > 1:
            return "Multiple" # Return generic value so we can check the state manually 
        elif len(possible_matches) == 0:
            return "None"
        else:
            return possible_matches[0][0] # Return the state that we matched 

    def save_gender_age_to_csv(self): # Save gender and age to a CSV
        self.cur.execute("SELECT face_data FROM upwork_face_data_unitedstates_allskills_2017_12_12;")
        
        age_ranges = [19, 24, 34, 44, 54, 64, 100] # Cutoff points for age
        
        with open(self.face_data_file_name, 'w') as gender_age_csv:
            fieldnames = ['user_count', 'gender', 'age', 'age_range_id']
            writer = csv.DictWriter(gender_age_csv, fieldnames=fieldnames)
            writer.writeheader()
            
            for user in self.cur:
                try: 
                    gender = user[0][0]['faceAttributes']['gender']
                    age = round(user[0][0]['faceAttributes']['age'])
                    age_range_id = 0
                    
                    for number in age_ranges:
                        if age <= number:
                            age_range_id = age_ranges.index(number)
                            break
                    
                    writer.writerow({'user_count': self.user_count,
                                    'gender': gender, 'age': age, 'age_range_id': age_range_id})
                    print "Finished writing data for {0}".format(self.user_count)

                except Exception as error:
                    print "There was an error at {0}".format(self.user_count)
                    print error 
                    writer.writerow({'user_count': self.user_count,
                                    'gender': "error", 'age': "error", 'age_range_id': "error"})

                self.user_count += 1
        
    def save_face_data(self): # Save face data of users' whose photos have been pushed to GitHub 
        self.face_api_key = raw_input("Paste the Face API subscription key here: ")
        worker_id_array = []
        
        self.cur.execute("SELECT detailed_info FROM upwork_unitedstates_allskills_2017_12_12;")
        
        for user in cur:
            worker_id = user[0]["ciphertext"]
            worker_id_array.append(worker_id)
        
        for worker in worker_id_array:
            try: 
                user_count = self.user_count
                worker_id = worker
                face_data = self.get_face_data()
                self.cur.execute("INSERT INTO upwork_face_data_unitedstates_allskills_2017_12_12 (user_count, worker_id, face_data) VALUES (%s, %s, %s);", 
                                 [user_count, worker_id, psycopg2.extras.Json(face_data)])
                self.conn.commit()
                print "Done with user {0}".format(user_count)
            
            except psycopg2.IntegrityError: # To prevent duplicate user_count from being added to the database
                self.conn.rollback()
                
            except Exception as err:
                user_count = self.user_count
                worker_id = "error"
                face_data = []
                self.cur.execute("INSERT INTO upwork_face_data_unitedstates_allskills_2017_12_12 (user_count, worker_id, face_data) VALUES (%s, %s, %s);", 
                                 [user_count, worker_id, psycopg2.extras.Json(face_data)])
                self.conn.commit()
                print "Ran into some error saving face data at {0}".format(self.user_count)
            
            self.user_count += 1
        
    def get_face_data(self):
        if (self.user_count%10 == 0): # Set timeout for rate limiting; paid subscription up to 10 per second 
            time.sleep(0.5)
        
        try: 
            print "Recognizing Face Number " + str(self.user_count)
            raw_face_data = self.recognize_faces()
            return raw_face_data
            
        except Exception as err:
            print err
            self.photo_log.write("Ran into an error in identifying gender at {0}: {1}".format(self.user_count, err) + "\n")
            self.photo_log.flush()
            return [] # Return empty JSON  
        
    def recognize_faces(self):
        # Replace the subscription_key string value with your valid subscription key.
        subscription_key = self.face_api_key 

        # Replace or verify the region.
        uri_base = 'eastus2.api.cognitive.microsoft.com'

        # Request headers.
        headers = {
            'Content-Type': 'application/json',
            'Ocp-Apim-Subscription-Key': subscription_key,
        }

        # Request parameters.
        params = urllib.urlencode({
            'returnFaceId': 'true',
            'returnFaceLandmarks': 'false',
            'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
        })

        # The URL of a JPEG image to analyze.
        beg_body = "{'url':"
        end_body = "}"
        path = self.github_photo_path + str(self.user_count) + ".jpg"
        body = beg_body + '"' + path + '"' + end_body
        
        try:
            # Execute the REST API call and get the response.
            conn = httplib.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
            conn.request("POST", "/face/v1.0/detect?%s" % params, body, headers)
            response = conn.getresponse()
            data = response.read()
            parsed = json.loads(data)
            return parsed
            
        except Exception as e:
            print("[Errno {0}] {1}".format(e.errno, e.strerror))
            return [] # Return empty JSON 
            
        '''
        # 'data' contains the JSON data. The following formats the JSON data for display.
        parsed = json.loads(data)
        print ("Response:")
        print (json.dumps(parsed, sort_keys=True, indent=2))
        conn.close()
        '''
    
    def calculate_work_experience(self, user):
        total_experience = 0
        work_experience_array = []
        
        try: 
            work_experience_list = user[0]["experiences"]["experience"]
            
        except:
            return 0
        
        if (type(work_experience_list) is list):
            count = 0
            total_overlap = 0
            

            for experience in work_experience_list:
                start_date = experience["exp_from"]
                end_date = experience["exp_to"]
                if (end_date == "Present"):
                    end_date = self.present_date
                start_datetime = datetime.datetime.strptime(start_date, "%m/%Y")
                end_datetime = datetime.datetime.strptime(end_date, "%m/%Y")

                if (start_datetime > end_datetime): #If the dates are somehow flipped
                    return "error"
                else: 
                    work_experience_array.append([start_datetime, end_datetime])
                    experience_duration = (end_datetime - start_datetime).days
                    total_experience += experience_duration

            while (count < len(work_experience_array) - 1): # Check for overlap in work experience 
                # While current index + offset < len
                offset = 1

                while (offset + count < len(work_experience_array)): 

                    # Check which of the two end dates are the newest and assign appropriate values
                    end_date1 = work_experience_array[count][1]
                    end_date2 = work_experience_array[count + offset][1]

                    if (end_date2 <= end_date1):
                        start_newer = work_experience_array[count][0]
                        end_newer = work_experience_array[count][1]
                        start_older = work_experience_array[count + offset][0]
                        end_older = work_experience_array[count + offset][1] 

                    elif (end_date2 > end_date1):
                        start_newer = work_experience_array[count + offset][0]
                        end_newer = work_experience_array[count + offset][1]
                        start_older = work_experience_array[count][0]
                        end_older = work_experience_array[count][1]

                    # Check for patterns as usual 
                    if (end_newer == end_older): # Pattern 1
                        newer_duration = (end_newer - start_newer).days
                        older_duration = (end_older - start_older).days
                        overlap = min(newer_duration, older_duration)
                        total_overlap += overlap 
                        offset += 1

                    elif(start_older == start_newer): # Pattern 2
                        overlap = (end_older - start_older).days
                        total_overlap += overlap 
                        offset += 1

                    elif(start_newer < start_older and end_newer > end_older): # Pattern 3
                        overlap = (end_older - start_older).days
                        total_overlap += overlap 
                        offset += 1

                    elif(start_newer > start_older and end_newer > end_older and end_older > start_newer): # Pattern 4
                        overlap = (end_older - start_newer).days
                        total_overlap += overlap 
                        offset += 1

                    else:
                        offset += 1

                count += 1

            #print "Total experience before overlap in months: {0}".format(total_experience/30)
            #print "Total overlap in months: {0}".format(total_overlap/30)
            total_experience = (total_experience - total_overlap)/30
            #print "Total experience after overlap in months: {0}".format(total_experience)
            total_experience = total_experience/12 # Convert to years
            #print "Total experience after overlap in years: {0}".format(total_experience)

            return total_experience
            
        elif (type(work_experience_list) is dict):
            start_date = work_experience_list["exp_from"]
            end_date = work_experience_list["exp_to"]
            if (end_date == "Present"):
                end_date = self.present_date
            start_datetime = datetime.datetime.strptime(start_date, "%m/%Y")
            end_datetime = datetime.datetime.strptime(end_date, "%m/%Y")
            #print "Start date: {0}, End_date: {1}".format(start_datetime, end_datetime)
            
            if (start_datetime >= end_datetime):
                return "error"
                print "the dates were flipped or too short time"
            
            total_experience = ((end_datetime - start_datetime).days)/30
            total_experience = total_experience/12 # Convert to years
            
            #print "Single experience: {0}".format(total_experience)
            return total_experience
        
        else: 
            return 0
    
    def show_degree(self, user):
        try: 
            ed_history = user[0]["education"]["institution"]
            if(ed_history == ""):
                return "No edu listed"
            elif(type(ed_history) == dict):
                return ed_history["ed_degree"]
            elif(type(ed_history) == list):
                degree_list = []
                for ed_experience in user[0]["education"]["institution"]:
                    if (ed_experience["ed_degree"] == ""):
                        degree_list.append("none")
                    elif (ed_experience["ed_degree"] != ""):
                        degree_list.append(ed_experience["ed_degree"])
                return degree_list
            else:
                return "Error: Some other error"
                
        except:
            return "Error: No education"
    

    def calculate_education(self, user):
        try: 
            ed_history = user[0]["education"]["institution"]
            if(ed_history == ""):
                return "No history"
            elif(type(ed_history) == dict):
                # Debugging: Check for high schools, then look at degree
                if (re.search("high school", ed_history["ed_school"].lower())):
                    self.high_school_degrees.append([ed_history["ed_school"], ed_history["ed_comment"], ed_history["ed_degree"]])
                # Look for highest education based on degree title, as normal 
                if (ed_history["ed_degree"] == ""):
                    highest_education = self.check_highest_education("")
                elif (ed_history["ed_degree"] != ""):
                    highest_education = self.check_highest_education(ed_history["ed_degree"])
                    if (highest_education == "Other"):
                        self.other_info.append([ed_history["ed_school"], ed_history["ed_comment"], ed_history["ed_degree"]])
                    elif (highest_education == "Diploma"):
                        self.diploma_info.append([ed_history["ed_school"], ed_history["ed_comment"], ed_history["ed_degree"]])
                    elif (highest_education == "Ambiguous"):
                        self.ambiguous_info.append([ed_history["ed_school"], ed_history["ed_comment"], ed_history["ed_degree"]])               
                # Check for highest level of degree
                if(highest_education == "Other"): # Don't count anything we can't identify
                    return "Ambiguous"
                elif(highest_education == "None specified"): # Don't count anything that isn't specified 
                    return "None specified"
                elif(highest_education == "Empty"):
                    return "Empty"
                elif(highest_education == "Doctorate"):
                    return "Doctorate"
                elif(highest_education == "Professional"):
                    return "Professional"
                elif(highest_education == "Master"):
                    return "Master"
                elif(highest_education == "Bachelor"):
                    return "Bachelor"
                elif(highest_education == "Associate"):
                    return "Associate"
                elif(highest_education == "High School"):
                    return "High School"
                else:
                    return "None"
      
            elif(type(ed_history) == list):
                ed_list = []
                for ed_experience in user[0]["education"]["institution"]:
                    # Debugging: Check for high schools, then look at degree
                    if (re.search("high school", ed_experience["ed_school"].lower())):
                        self.high_school_degrees.append([ed_experience["ed_school"], ed_experience["ed_comment"], ed_experience["ed_degree"]])
                    if (ed_experience["ed_degree"] == ""):
                        self.schools_no_degree.append(ed_experience["ed_school"])
                        highest_education = self.check_highest_education("")
                        ed_list.append(highest_education)
                    elif (ed_experience["ed_degree"] != ""):
                        highest_education = self.check_highest_education(ed_experience["ed_degree"])
                        if (highest_education == "None specified"):
                            self.none_specified_info.append([ed_experience["ed_school"], ed_experience["ed_comment"], ed_experience["ed_degree"]])
                        elif (highest_education == "Empty"):
                            self.empty_info.append([ed_experience["ed_school"], ed_experience["ed_comment"], ed_experience["ed_degree"]])
                        elif (highest_education == "Other"):
                            self.other_info.append([ed_experience["ed_school"], ed_experience["ed_comment"], ed_experience["ed_degree"]])
                        elif (highest_education == "Diploma"):
                            self.diploma_info.append([ed_experience["ed_school"], ed_experience["ed_comment"], ed_experience["ed_degree"]])
                        elif (highest_education == "Ambiguous"):
                            self.ambiguous_info.append([ed_experience["ed_school"], ed_experience["ed_comment"], ed_experience["ed_degree"]])
                        ed_list.append(highest_education)
                # Check for highest level of degree
                if("Other" in ed_list): # Don't count anything we can't identify
                    return "Other"
                elif("Ambiguous" in ed_list):
                    return "Ambiguous"
                elif("None specified" in ed_list): # Don't count anything that isn't specified 
                    return "None specified"
                elif ("Doctorate" in ed_list):
                    return "Doctorate"
                elif("Professional" in ed_list):
                    return "Professional"
                elif("Master" in ed_list):
                    return "Master"
                elif("Bachelor" in ed_list):
                    return "Bachelor"
                elif("Associate" in ed_list):
                    return "Associate"
                
                # Added diploma condition for debugging purposes 
                elif("Diploma" in ed_list):
                    return "Diploma"
                elif("High School" in ed_list):
                    return "High School"
                else:
                    return "None"
            else:
                return "Error: Some other error"
            
            print other_info
            print diploma_info
                
        except:
            return "None"
        
    
    def check_highest_education(self, degree):
        # Checks for the highest level of education attained by user (DO NOT CHANGE ORDER!) 
        
        # Professional degree
        if (re.search('d\.c|d\.c\.m|d\.d\.s|d\.m\.d|ll\.b|ll\.m|l\.l\.m|l\.l\.b|j\.d\.|m\.d\.|o\.d\.|d\.o\.|pharm\.d', degree.lower()) 
            or re.search('d\.p\.m|d\.p|pod\.d|m\.div|m\.h\.l|b\.d|ordination|d\.v\.m| law', degree.lower())
            or re.search('DC|DCM|DDS|DMD|LLB|LLM|LLB|JD|MD|OD|DO|PharmD|Ed\.D|EdD|CFA', degree)
            or re.search('DPM|DP|PodD|MDiv|MHL|BD|DVM', degree)):
            return "Professional"
        
        # PhD
        elif (re.search('Ph\.D|PhD', degree)
             or re.search('doctor|licentiate', degree.lower())): # Doctorate placed here in case MDs are also called doctors 
            return "Doctorate"
  
        # Masters
        elif (re.search('M\.B\.A|MBA|M\.A|M\.S|M\.Sc|M\.E|MA|MS|MSc|ME|M\.A\.Ed|MAEd|MST|M\.S\.T|MEd|M\.Ed|MPA|M\.P\.A', degree)
             or re.search('master|engineer\'s degree', degree.lower())
             or re.search('MFA|M\.F\.A|MSC|MCS', degree)):
            return "Master"

        # Bachelors
        elif (re.search('B\.A|B\.S|B\.Sc|BA|BS|BSc|BFA|AB|A\.B|BPharm|B\.Pharm', degree)
             or re.search('bachelor|bpharm|bachlor|batchelor', degree.lower())):
            return "Bachelor"
        
        # Associate's Degree
        elif (re.search('AA|AS|A\.A|A\.S|A\.A\.S|AAS', degree)
             or re.search('associate|foundation', degree.lower())):
            return "Associate"
        
        # Certificate
        elif (re.search('certificate', degree.lower())):
            return "Certificate"

        # High School
        elif (re.search('high school', degree.lower())):
            return "High School"
        
        # None
        elif (re.search('none', degree.lower())):
            return "None specified"
        
        # Empty
        elif (degree == ""):
            return "Empty"
        
        # Other
        elif (re.search('other', degree.lower())):
            return "Other"
        
        # Diploma
        elif (re.search('diploma', degree.lower())):
            return "Diploma"

        else:
            self.list_of_ambiguous_degrees.append(degree)
            return "Ambiguous" # We couldn't recognize the degree 
    
    def encode_category(self, item, id_list):
        if not(item in id_list):
            id_list.append(item)
        
        category_index = id_list.index(item)
        return category_index
        
    
    def identify_job_category(self, user):
        try:
            all_job_categories = user[0]["dev_job_categories_v2"]["dev_job_categories_v"]
            
        except:
            job_category = "none"
            return job_category
        
        try:
            job_category = all_job_categories[0]["groups"]["group"]["name"] # Returns general job category
        
        except:
            job_category = all_job_categories["groups"]["group"]["name"]
        
        return job_category

myObject = UpworkDataFormatter()
myObject.save_all_to_csv()

Finished writing data for 1
Finished writing data for 2
Finished writing data for 3
Finished writing data for 4
Finished writing data for 5
Finished writing data for 6
Finished writing data for 7
Finished writing data for 8
Finished writing data for 9
Ran into some error at user 10
Finished writing data for 10
Finished writing data for 11
Ran into some error at user 12
Finished writing data for 12
Ran into some error at user 13
Finished writing data for 13
Finished writing data for 14
Ran into some error at user 15
Finished writing data for 15
Finished writing data for 16
Ran into some error at user 17
Finished writing data for 17
Ran into some error at user 18
Finished writing data for 18
Finished writing data for 19
Finished writing data for 20
Ran into some error at user 21
Finished writing data for 21
Ran into some error at user 22
Finished writing data for 22
Ran into some error at user 23
Finished writing data for 23
Finished writing data for 24
Finished writing data for 25
Finis

Finished writing data for 297
Finished writing data for 298
Finished writing data for 299
Finished writing data for 300
Finished writing data for 301
Finished writing data for 302
Finished writing data for 303
Finished writing data for 304
Finished writing data for 305
Finished writing data for 306
Finished writing data for 307
Finished writing data for 308
Finished writing data for 309
Finished writing data for 310
Ran into some error at user 311
Finished writing data for 311
Finished writing data for 312
Finished writing data for 313
Ran into some error at user 314
Finished writing data for 314
Finished writing data for 315
Finished writing data for 316
Finished writing data for 317
Finished writing data for 318
Finished writing data for 319
Finished writing data for 320
Finished writing data for 321
Ran into some error at user 322
Finished writing data for 322
Finished writing data for 323
Finished writing data for 324
Finished writing data for 325
Finished writing data for 326
Fini

Finished writing data for 581
Ran into some error at user 582
Finished writing data for 582
Finished writing data for 583
Finished writing data for 584
Ran into some error at user 585
Finished writing data for 585
Ran into some error at user 586
Finished writing data for 586
Finished writing data for 587
Ran into some error at user 588
Finished writing data for 588
Finished writing data for 589
Finished writing data for 590
Finished writing data for 591
Finished writing data for 592
Ran into some error at user 593
Finished writing data for 593
Finished writing data for 594
Ran into some error at user 595
Finished writing data for 595
Finished writing data for 596
Finished writing data for 597
Finished writing data for 598
Finished writing data for 599
Finished writing data for 600
Ran into some error at user 601
Finished writing data for 601
Finished writing data for 602
Ran into some error at user 603
Finished writing data for 603
Finished writing data for 604
Finished writing data fo

Finished writing data for 874
Finished writing data for 875
Finished writing data for 876
Finished writing data for 877
Finished writing data for 878
Finished writing data for 879
Finished writing data for 880
Ran into some error at user 881
Finished writing data for 881
Finished writing data for 882
Finished writing data for 883
Finished writing data for 884
Finished writing data for 885
Finished writing data for 886
Finished writing data for 887
Ran into some error at user 888
Finished writing data for 888
Finished writing data for 889
Finished writing data for 890
Finished writing data for 891
Ran into some error at user 892
Finished writing data for 892
Finished writing data for 893
Finished writing data for 894
Finished writing data for 895
Finished writing data for 896
Finished writing data for 897
Finished writing data for 898
Finished writing data for 899
Finished writing data for 900
Ran into some error at user 901
Finished writing data for 901
Finished writing data for 902
Fi

Finished writing data for 1145
Finished writing data for 1146
Finished writing data for 1147
Finished writing data for 1148
Finished writing data for 1149
Finished writing data for 1150
Finished writing data for 1151
Finished writing data for 1152
Finished writing data for 1153
Finished writing data for 1154
Finished writing data for 1155
Finished writing data for 1156
Finished writing data for 1157
Ran into some error at user 1158
Finished writing data for 1158
Finished writing data for 1159
Finished writing data for 1160
Finished writing data for 1161
Ran into some error at user 1162
Finished writing data for 1162
Finished writing data for 1163
Finished writing data for 1164
Finished writing data for 1165
Finished writing data for 1166
Ran into some error at user 1167
Finished writing data for 1167
Finished writing data for 1168
Finished writing data for 1169
Finished writing data for 1170
Finished writing data for 1171
Finished writing data for 1172
Finished writing data for 1173
Fi

...

Ran into some error at user 52423
Finished writing data for 52423
Finished writing data for 52424
Finished writing data for 52425
Finished writing data for 52426
Finished writing data for 52427
Finished writing data for 52428
Finished writing data for 52429
Finished writing data for 52430
Finished writing data for 52431
Finished writing data for 52432
Finished writing data for 52433
Finished writing data for 52434
Finished writing data for 52435
Finished writing data for 52436
Finished writing data for 52437
Finished writing data for 52438
Finished writing data for 52439
Finished writing data for 52440
Finished writing data for 52441
Finished writing data for 52442
Finished writing data for 52443
Finished writing data for 52444
Finished writing data for 52445
Finished writing data for 52446
Finished writing data for 52447
Finished writing data for 52448
Finished writing data for 52449
Finished writing data for 52450
Ran into some error at user 52451
Finished writing data for 52451
Ran 

Finished writing data for 53076
Finished writing data for 53077
Finished writing data for 53078
Finished writing data for 53079
Finished writing data for 53080
Finished writing data for 53081
Ran into some error at user 53082
Finished writing data for 53082
Finished writing data for 53083
Finished writing data for 53084
Finished writing data for 53085
Ran into some error at user 53086
Finished writing data for 53086
Finished writing data for 53087
Finished writing data for 53088
Ran into some error at user 53089
Finished writing data for 53089
Ran into some error at user 53090
Finished writing data for 53090
Finished writing data for 53091
Finished writing data for 53092
Finished writing data for 53093
Finished writing data for 53094
Finished writing data for 53095
Finished writing data for 53096
Finished writing data for 53097
Finished writing data for 53098
Finished writing data for 53099
Finished writing data for 53100
Finished writing data for 53101
Finished writing data for 53102


Finished writing data for 53694
Finished writing data for 53695
Finished writing data for 53696
Finished writing data for 53697
Finished writing data for 53698
Finished writing data for 53699
Finished writing data for 53700
Finished writing data for 53701
Finished writing data for 53702
Finished writing data for 53703
Ran into some error at user 53704
Finished writing data for 53704
Ran into some error at user 53705
Finished writing data for 53705
Finished writing data for 53706
Finished writing data for 53707
Finished writing data for 53708
Finished writing data for 53709
Finished writing data for 53710
Finished writing data for 53711
Ran into some error at user 53712
Finished writing data for 53712
Finished writing data for 53713
Finished writing data for 53714
Finished writing data for 53715
Finished writing data for 53716
Finished writing data for 53717
Finished writing data for 53718
Finished writing data for 53719
Finished writing data for 53720
Finished writing data for 53721
Fi

Finished writing data for 54315
Ran into some error at user 54316
Finished writing data for 54316
Finished writing data for 54317
Finished writing data for 54318
Finished writing data for 54319
Ran into some error at user 54320
Finished writing data for 54320
Finished writing data for 54321
Finished writing data for 54322
Finished writing data for 54323
Ran into some error at user 54324
Finished writing data for 54324
Finished writing data for 54325
Finished writing data for 54326
Finished writing data for 54327
Finished writing data for 54328
Ran into some error at user 54329
Finished writing data for 54329
Finished writing data for 54330
Finished writing data for 54331
Finished writing data for 54332
Finished writing data for 54333
Finished writing data for 54334
Finished writing data for 54335
Finished writing data for 54336
Finished writing data for 54337
Finished writing data for 54338
Finished writing data for 54339
Finished writing data for 54340
Finished writing data for 54341


Ran into some error at user 55018
Finished writing data for 55018
Finished writing data for 55019
Finished writing data for 55020
Finished writing data for 55021
Finished writing data for 55022
Ran into some error at user 55023
Finished writing data for 55023
Finished writing data for 55024
Finished writing data for 55025
Finished writing data for 55026
Finished writing data for 55027
Ran into some error at user 55028
Finished writing data for 55028
Ran into some error at user 55029
Finished writing data for 55029
Finished writing data for 55030
Finished writing data for 55031
Finished writing data for 55032
Finished writing data for 55033
Finished writing data for 55034
Finished writing data for 55035
Finished writing data for 55036
Finished writing data for 55037
Finished writing data for 55038
Finished writing data for 55039
Finished writing data for 55040
Finished writing data for 55041
Finished writing data for 55042
Finished writing data for 55043
Finished writing data for 55044
